<a href="https://colab.research.google.com/github/kimhwijin/TensorflowWithKeras/blob/master/RNN/RNN_GRU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import tensorflow as tf
import numpy as np
import re
import shutil

CHECKPOINT_DIR = 'drive/MyDrive/Colab Notebooks/models/RNN/alice'
DATA_DIR = 'drive/MyDrive/Datasets/alice'


In [6]:
def download_and_read(urls):
  texts = []
  for i, url in enumerate(urls):
    p = tf.keras.utils.get_file("ex1-{:d}.txt".format(i), url, cache_dir=".")
    text = open(p, "r").read()
    text = text.replace("\ufeff", "")
    text = text.replace("\n", " ")
    text = re.sub(r'\s+', " ", text)
    texts.extend(text)
  return texts

texts = download_and_read([
                           "https://www.gutenberg.org/cache/epub/28885/pg28885.txt",
                           "https://www.gutenberg.org/files/12/12-0.txt"
])

vocab = sorted(set(texts))
print("vocab size:{:d}".format(len(vocab)))

#어휘 문자에서 정수로 매핑
char2idx = {c:i for i, c in enumerate(vocab)}
idx2char = {i:c for c , i in char2idx.items()}

#텍스트 수치화
texts_as_ints = np.array([char2idx[c] for c in texts])
data = tf.data.Dataset.from_tensor_slices(texts_as_ints)

#예측하기 전에 보여줄 문자 개수
seq_length = 100
sequences = data.batch(seq_length + 1, drop_remainder=True)

def split_train_labels(sequence):
  input_seq = sequence[0:-1]
  output_seq = sequence[1:]
  return input_seq, output_seq

sequences = sequences.map(split_train_labels)
batch_size = 64
steps_per_epoch = len(texts) // seq_length // batch_size
dataset = sequences.shuffle(10000).batch(batch_size, drop_remainder=True)


vocab size:92


In [10]:
class CharGenModel(tf.keras.Model):
  def __init__(self, vocab_size, num_timesteps, embedding_dim, rnn_output_dim, **kwargs):
    super(CharGenModel, self).__init__(**kwargs)
    self.embedding_layer = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.rnn_layer = tf.keras.layers.GRU(num_timesteps, recurrent_initializer="glorot_uniform", recurrent_activation="sigmoid", stateful=True, return_sequences=True)
    self.dense_layer = tf.keras.layers.Dense(vocab_size)
  def call(self, x):
    x = self.embedding_layer(x)
    x = self.rnn_layer(x)
    x = self.dense_layer(x)
    return x

vocab_size = len(vocab)
embedding_dim = 256
rnn_output_dim = 1024

model = CharGenModel(vocab_size, seq_length, embedding_dim, rnn_output_dim)
model.build(input_shape=(batch_size, seq_length))

In [ ]:
def loss(labels, predictions):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, predictions, from_logits=True)
model.compile(optimizer=tf.optimizers.Adam(), loss=loss)W